# (a)

Given a training set $\{x^{(i)},y^{(i)}\}_{i=1}^{m}$ with $y^{(i)}\in\{0,1\}$, we read in the class notes (1), p.18 how to derive the log-likelihood and its partial derivatives. Setting the partials to zero, we have

$$
0=\frac{\partial \ell(\theta)}{\partial \theta_j} = \sum_{i=1}^{m}(y^{(i)} - h_{\theta}(x^{(i)}))x_{j}^{(i)}
$$

where $h_\theta$ is defined as

$$
h_\theta(x^{(i)})\equiv\frac{1}{1 + e^{-\theta^T x^{(i)}}}\equiv P(y^{(i)} = 1 | x^{(i)}; \theta)
$$

Recall that for the intercept term, we set $x_0^{(i)}=1$ for all $i=1,...,m$. Hence

$$
0=\frac{\partial \ell(\theta)}{\partial \theta_0} = \sum_{i=1}^{m}(y^{(i)} - h_{\theta}(x^{(i)}))x_{0}^{(i)} = \sum_{i=1}^{m}(y^{(i)} - h_{\theta}(x^{(i)}))
$$

and 

$$\begin{align*}
\sum_{i=1}^{m}\boldsymbol{1} \{y^{(i)} = 1\} = \sum_{i=1}^{m}y^{(i)} = \sum_{i=1}^{m}h_\theta(x^{(i)}) = \sum_{i=1}^{m} P(y^{(i)}=1|x^{(i)};\theta)
\end{align*}$$

Note that $h_\theta(x^{(i)})\in(0,1)$ for all $i\in\{1,...,m\}$. Hence $I_{0,1}=\big\{i:i\in\{1,...,m\}\text{ and }h_\theta(x^{(i)})\in(0,1)\big\}=\{1,...,m\}$ and the above equality is equivalent to 

$$
\frac{\sum_{i \in I_{0, 1}} P(y^{(i)} = 1 | x^{(i)}; \theta)}{\left | \{i \in I_{0, 1} \} \right |} = \frac{\sum_{i \in I_{0, 1}} \boldsymbol{1}\{y^{(i)} = 1\}}{\left | \{i \in I_{0, 1} \} \right |}
$$

# (b)

Perfect calibration doesn't necessarily imply perfect accuracy. Let's take a counterexample.

$$
X=\begin{bmatrix}x_0^{(1)}&x_1^{(1)}\\x_0^{(2)}&x_1^{(2)}\end{bmatrix}=\begin{bmatrix}1&-1\\1&-1\end{bmatrix}
\quad\quad
y=\begin{bmatrix}y^{(1)}\\y^{(2)}\end{bmatrix}=\begin{bmatrix}1\\0\end{bmatrix}
\quad\quad
\theta=\begin{bmatrix}c\\c\end{bmatrix}
$$

where $c\in\mathbb{R}$ - see the section "Logistic Regression on X gives theta=(c,c)" below. Then for $i=1,2$, we have

$$
\theta^Tx^{(i)}=c\cdot1+c\cdot(-1)=0
\quad\text{and}\quad
P(y^{(i)} = 1 | x^{(i)};\theta)=h_\theta(x^{(i)})=\frac{1}{1+e^{-0}}=\frac{1}{2}
$$

and for any $0\leq a<\frac{1}{2}<b\leq1$, we have

$$
\frac{\sum_{i \in I_{a, b}} P(y^{(i)} = 1 | x^{(i)}; \theta)}{\left | \{i \in I_{a, b} \} \right |} = \frac{1}{2} = \frac{\sum_{i \in I_{a, b}} \boldsymbol{1}\{y^{(i)} = 1\}}{\left | \{i \in I_{a, b} \} \right |}
$$

and if $b>a\geq\frac{1}{2}$ or $a<b\leq\frac{1}{2}$, then $\lvert\{i \in I_{a, b} \}\rvert=0$. Hence this model is perfectly calibrated. But it's not perfectly accurate: $\theta^Tx^{(2)}=0$ so we predict $y^{(2)}=1$ which is wrong.



In [1]:
# for the given counterexample, we see that if theta is of the form [c, c], then the
# sum of the probabilities equals the sum of the labels

import numpy as np

x1=np.array([1, -1])
x2=np.array([1, -1])
y=np.array([1,0])
theta=np.array([-101, -101])

h=lambda theta, x: 1./(1 + np.exp(-theta.dot(x)))

h1=h(theta=theta, x=x1)
h2=h(theta=theta, x=x2)
print("h1={}\th2={}".format(h1,h2))

lhs=h1+h2
rhs=np.sum(y)
print("lhs={}\trhs={}".format(lhs, rhs))

count_lhs_equals_rhs = 0
for i in range(100):
    c = np.random.rand()
    theta = np.array([c, c])
    h1=h(theta=theta, x=x1)
    h2=h(theta=theta, x=x2)
    lhs=h1+h2
    if np.abs(lhs-rhs)<1e-7: count_lhs_equals_rhs+=1

print("Accuracy that lhs == rhs={}".format(count_lhs_equals_rhs/100))

h1=0.5	h2=0.5
lhs=1.0	rhs=1
Accuracy that lhs == rhs=1.0


In [2]:
default_init_theta = np.array([-1.1, 3.1])

# this is the logistic regression given in the Vectorization discussion

def sigmoid(z):
    signal = np.clip(z, -500, 500 )
    return 1. / (1 + np.exp(-signal))

def h(theta, X):
    return sigmoid(np.matmul(X, theta))

def calc_grad(theta, X, y):
    diff = h(theta, X) - y
    diff = np.reshape(diff, [1, -1])
    dt = np.matmul(diff, X)
    return np.squeeze(dt)
    
def train(X_train, y_train, alpha, init_theta=default_init_theta):
    theta=init_theta
    i = 0
    while True:
        i+=1
        prev_theta = theta
        grad = calc_grad(theta, X_train, y_train)
        theta = theta - alpha * grad
        error = np.linalg.norm(prev_theta - theta)
        #if i%100 == 0: print("iters={}  grad={}  error={}  theta={}  prev_theta={}".format(i, grad, error, theta, prev_theta))
        if error < 1e-15:
            break
    return theta, i

In [3]:
# this is logistic regression that I derived and implemented in Binary-Logistic-Regression

def llc(xt, y, regularization=0, theta=0):
    return 1/xt.shape[0] * (np.sum(np.log(1 + np.exp(np.clip(xt, -700, 700)))) - np.dot(y,xt)) \
        + (regularization * (np.linalg.norm(theta)**2))

def llg(X, xt, y, regularization=0, theta=0):
    return 1/xt.shape[0] * (sigmoid(xt) - y).dot(X) + (regularization * 2 * theta)

def log_likelihood_cost(X, theta, y, cost_or_grad='both', regularization=0):
    xt = X.dot(theta)
    th = theta if regularization else 0
    if cost_or_grad == 'both':
        return (llc(xt, y, regularization=regularization, theta=th), llg(X, xt, y, regularization=regularization, theta=th))
    if cost_or_grad == 'cost':
        return llc(xt, y, regularization=regularization, theta=th)
    if cost_or_grad == 'grad':
        return llg(X, xt, y, regularization=regularization, theta=th)

def logistic_regression(X, y, learning_rate, init_theta=default_init_theta, max_iters=800*1000, scale_lr_prop=0, regularization=0, prntiters=1000, tol=1e-7):
    theta= init_theta
    thetas, thetas_sample, errors, losses = [], [], [], []
    i = 0
    while True:
        i += 1
        prev_theta = theta
        cost, grad = log_likelihood_cost(X, theta, y, regularization=regularization, cost_or_grad='both')
        theta = theta - learning_rate * grad
        thetas.append(theta)
        errors.append(np.linalg.norm(prev_theta - theta))
        losses.append(cost)
        if scale_lr_prop: learning_rate *= scale_lr_prop
        if i % prntiters == 0:
            print('iters={} theta={} error={} loss={}'.format(i, np.mean(theta), errors[-1], losses[-1]))
            thetas_sample.append(theta)
        if errors[-1] < tol:
            break
        if i == max_iters:
            break
    return np.array(thetas), i

In [4]:
# Logistic Regression on X gives theta=(c,c)

X=np.array([[1, -1],[1, -1]])
y=np.array([1,0])

thetaw, iters = train(X,y,1)
print("thetaw={} iters={}".format(thetaw, iters))
thetaw, iters = logistic_regression(X, y, learning_rate=2, tol=1e-15)
print("thetaw={} iters={}".format(thetaw[-1], iters))

init_theta=np.array([117., -319.])

thetaw, iters = train(X,y,1, init_theta=init_theta)
print("thetaw={} iters={}".format(thetaw, iters))
thetaw, iters = logistic_regression(X, y, learning_rate=2, init_theta=init_theta, tol=1e-15)
print("thetaw={} iters={}".format(thetaw[-1], iters))

count_t0_equals_t1 = 0
for i in range(100):
    it = np.random.randn(2)*10
    t,i = train(X,y,1, init_theta=it)
    if np.abs(t[0]-t[1])<1e-7: count_t0_equals_t1+=1

print("Accuracy that theta[0] == theta[1]={}".format(count_t0_equals_t1/100))

thetaw=[ 1.  1.] iters=6
thetaw=[ 1.  1.] iters=6
thetaw=[-101. -101.] iters=222
thetaw=[-101. -101.] iters=222
Accuracy that theta[0] == theta[1]=1.0


Conversely:

__Proposition__ Perfect accuracy implies imperfect calibration unless $P(y^{(i)} = 1 | x^{(i)}; \theta)$ equals zero or one for all $i=1,...,m$.

__Proof__ Perfect accuracy means that for all $i=1,...,m$, we have

$$\begin{gather}
y^{(i)}=1\iff\theta^Tx^{(i)}\geq0\iff P(y^{(i)} = 1 | x^{(i)}; \theta)=h_\theta(x^{(i)})\geq\frac{1}{2}\\
y^{(i)}=0\iff\theta^Tx^{(i)}<0\iff P(y^{(i)} = 1 | x^{(i)}; \theta)=h_\theta(x^{(i)})<\frac{1}{2}\\
\end{gather}$$

Fix some $a<b\in(0,1)$. First suppose that $0<a<b<\frac{1}{2}$. Then

$$
\frac{\sum_{i \in I_{a, b}} P(y^{(i)} = 1 | x^{(i)}; \theta)}{\left | \{i \in I_{a, b} \} \right |} > \frac{a\left | \{i \in I_{a, b} \} \right |}{\left | \{i \in I_{a, b} \} \right |} = a > 0 = \frac{\sum_{i \in I_{a, b}} \boldsymbol{1}\{y^{(i)} = 1\}}{\left | \{i \in I_{a, b} \} \right |}
$$

The first inequality follows because $P(y^{(i)} = 1 | x^{(i)}; \theta)>a$ for all $i\in I_{a,b}=\big\{i:i\in\{1,...,m\}\text{ and }h_\theta(x^{(i)})\in(a,b)\big\}$.

The last equality follows because $P(y^{(i)} = 1 | x^{(i)}; \theta)<b<\frac{1}{2}$ for all $i\in I_{a,b}=\big\{i:i\in\{1,...,m\}\text{ and }h_\theta(x^{(i)})\in(a,b)\big\}$. Hence perfect accuracy implies that $y^{(i)}=0$ for all $i\in I_{a,b}$.

Similarly suppose that $\frac{1}{2}<a<b<1$. Then

$$
\frac{\sum_{i \in I_{a, b}} P(y^{(i)} = 1 | x^{(i)}; \theta)}{\left | \{i \in I_{a, b} \} \right |} < \frac{b\left | \{i \in I_{a, b} \} \right |}{\left | \{i \in I_{a, b} \} \right |} = b < 1 = \frac{\sum_{i \in I_{a, b}} \boldsymbol{1}\{y^{(i)} = 1\}}{\left | \{i \in I_{a, b} \} \right |}
$$

The first inequality follows because $P(y^{(i)} = 1 | x^{(i)}; \theta)<b$ for all $i\in I_{a,b}=\big\{i:i\in\{1,...,m\}\text{ and }h_\theta(x^{(i)})\in(a,b)\big\}$.

The last equality follows because $P(y^{(i)} = 1 | x^{(i)}; \theta)>a>\frac{1}{2}$ for all $i\in I_{a,b}=\big\{i:i\in\{1,...,m\}\text{ and }h_\theta(x^{(i)})\in(a,b)\big\}$. Hence perfect accuracy implies that $y^{(i)}=1$ for all $i\in I_{a,b}$.

However, if we have perfect accuracy and $P(y^{(i)} = 1 | x^{(i)}; \theta)$ equals zero or one for all $i=1,...,m$, then it must be that

$$\begin{gather}
y^{(i)}=1\iff P(y^{(i)} = 1 | x^{(i)}; \theta)=1\\
y^{(i)}=0\iff P(y^{(i)} = 1 | x^{(i)}; \theta)=0\\
\end{gather}$$

for all $i=1,...,m$. Hence the sum of the probabilities must equal the sum of the labels on any interval $(a,b)\subset[0,1]$.

$\blacksquare$

In the below sections starting with "logistic regression on dataset b" and culminating with "but we don't have perfect calibration", we present an example of logistic regression (using modified data from question 1) where we get perfect accuracy and imperfect calibration.

In [5]:
# logistic regression on dataset b

def add_intercept(X_):
    m, n = X_.shape
    X = np.zeros((m, n + 1))
    X[:, 0] = 1
    X[:, 1:] = X_
    return X

def load_data(filename):
    D = np.loadtxt(filename)
    Y = D[:, 0]
    X = D[:, 1:]
    return add_intercept(X), Y

Xb, yb = load_data('data_b.txt')
yb0 = yb.copy()
yb0[yb0==-1]=0
print("yb00={}  ybn1={}  yb01={}  yb1={}".format(np.sum(yb0==0), np.sum(yb==-1), np.sum(yb0==1), np.sum(yb==1)))

thetas_b, iters = logistic_regression(Xb, yb0, init_theta=np.zeros(Xb.shape[1]), learning_rate = 1, tol=1.5e-4, prntiters=30*1000)
print("theta for dataset b={}".format(thetas_b[-1]))

yb00=45  ybn1=45  yb01=55  yb1=55
iters=30000 theta=11.396397130921606 error=0.0007049968429742279 loss=0.046108675969032904
iters=60000 theta=14.620872501849346 error=0.0004596703601245677 loss=0.036586261243761554
iters=90000 theta=16.95286662197313 error=0.00035880194441672083 loss=0.03166951454270361
iters=120000 theta=18.84449918365077 error=0.000300442521457871 loss=0.028444732554851273
iters=150000 theta=20.46068797443075 error=0.0002612524439697501 loss=0.026093702877517444
iters=180000 theta=21.883770848432757 error=0.00023264724827086328 loss=0.02427212485286418
iters=210000 theta=23.162015535823915 error=0.00021062630965153873 loss=0.02280308305720837
iters=240000 theta=24.326649311436537 error=0.00019303506555163453 loss=0.021583949826092522
iters=270000 theta=25.399270990530145 error=0.00017859386272892503 loss=0.02055010584396541
iters=300000 theta=26.395558897713006 error=0.00016648707143217275 loss=0.019658376752513505
iters=330000 theta=27.327317494353633 error=0.00015

In [6]:
# these are the probabilities that are zero or one (or close to it)

h = sigmoid(Xb.dot(thetas_b[-1]))
cond_h01 = (h<1e-3) | (h>(1-1e-3))
print("sum_cond_h01={}\nh[cond_h01]={}".format(np.sum(cond_h01), h[cond_h01].round(3)))

sum_cond_h01=85
h[cond_h01]=[ 0.     1.     0.     0.     0.     1.     0.     1.     0.     0.     1.
  0.     0.     1.     1.     0.     0.     0.     0.     1.     1.     0.
  0.     1.     0.     1.     1.     1.     1.     1.     1.     0.     0.
  1.     0.     1.     1.     1.     1.     0.     0.     0.     0.     1.
  0.     1.     1.     1.     1.     1.     0.     1.     1.     0.999  1.
  1.     0.001  0.     1.     1.     1.     1.     0.     1.     0.     0.
  1.     0.     1.     1.     1.     0.     0.     0.     0.     1.     1.
  1.     1.     1.     0.     1.     1.     0.     1.   ]


In [7]:
# and these are the probabilities that aren't zero nor one

cond_hf = (h>1e-3) & (h<(1-1e-3))
print("sum_cond_hf={}\nh[cond_hf]={}".format(np.sum(cond_hf), h[cond_hf].round(3)))

sum_cond_hf=15
h[cond_hf]=[ 0.192  0.059  0.374  0.663  0.991  0.95   0.943  0.108  0.031  0.001
  0.035  0.743  0.996  0.92   0.001]


In [8]:
# we see that the accuracy is perfect

pred = (np.sign(h - 0.5) + 1) / 2
correct = np.sum(pred == yb0)
total = len(yb0)
print("correct={}  total={}  accuracy={}".format(correct, total, correct / total))

correct=100  total=100  accuracy=1.0


In [9]:
# but we don't have perfect calibration

a0, b0 = .03, .47
Iab0 = [i for i, h0 in enumerate((h>a0) & (h<b0)) if h0]
hf0 = h[Iab0].round(3)
yb0f0 = yb0[Iab0]
print("Iab0={}\nhf0={}\nyb0f0={}".format(Iab0, hf0, yb0f0))
print("{}=LHS != RHS={}\n\n".format((np.sum(hf0)/len(Iab0)).round(3), np.sum(yb0f0)/len(Iab0)))

a1, b1 = .53, .97
Iab1 = [i for i, h1 in enumerate((h>a1) & (h<b1)) if h1]
hf1 = h[Iab1].round(3)
yb0f1 = yb0[Iab1]
print("Iab1={}\nhf1={}\nyb0f1={}".format(Iab1, hf1, yb0f1))
print("{}=LHS != RHS={}".format((np.sum(hf1)/len(Iab1)).round(3), np.sum(yb0f1)/len(Iab1)))

Iab0=[1, 5, 7, 51, 66, 78]
hf0=[ 0.192  0.059  0.374  0.108  0.031  0.035]
yb0f0=[ 0.  0.  0.  0.  0.  0.]
0.133=LHS != RHS=0.0


Iab1=[26, 31, 39, 87, 97]
hf1=[ 0.663  0.95   0.943  0.743  0.92 ]
yb0f1=[ 1.  1.  1.  1.  1.]
0.844=LHS != RHS=1.0


In [10]:
# but it is calibrated for a=0, b=1, as the remark at the end of the question indicates is quaranteed

a2, b2 = 0., 1.
Iab2 = [i for i, h2 in enumerate((h>a2) & (h<b2)) if h2]
hf2 = h[Iab2].round(3)
yb0f2 = yb0[Iab2]
print("Iab2={}\nhf2={}\nyb0f2={}".format(Iab2, hf2, yb0f2))
print("{}=LHS ~= RHS={}".format((np.sum(hf2)/len(Iab2)).round(12), np.sum(yb0f2)/len(Iab2)))

Iab2=[0, 1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 24, 25, 26, 28, 29, 31, 32, 33, 34, 36, 37, 38, 39, 41, 43, 45, 46, 47, 48, 49, 51, 52, 53, 54, 56, 57, 58, 60, 61, 63, 64, 65, 66, 67, 69, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 82, 83, 84, 85, 87, 90, 91, 92, 93, 94, 96, 97, 98, 99]
hf2=[ 0.     0.192  0.     0.     0.059  0.     0.374  1.     0.     1.     0.
  0.     1.     0.     0.     1.     1.     0.     0.     0.     0.     1.
  0.     0.     0.663  0.     0.991  0.95   1.     1.     1.     1.     0.
  0.     0.943  0.     1.     1.     0.     0.     0.     0.     0.108  0.
  1.     1.     1.     1.     0.     1.     0.999  1.     0.001  0.     0.031
  0.001  1.     1.     0.     1.     0.     0.     1.     0.     0.035  1.
  1.     0.     0.     0.     0.     0.743  0.996  1.     1.     0.     1.
  0.     0.92   1.     0.001]
yb0f2=[ 0.  0.  0.  0.  0.  0.  0.  1.  0.  1.  0.  0.  1.  0.  0.  1.  1.  0.
  0.  0.  0.  1.  0.  0.  1.  0.  1.  

# (c)

Let $\lambda$ be the regularization parameter. When $\lambda$ is large, we find that calibration breaks down (see the sections below). This occurs because a larger $\lambda$ implies a smaller $\theta$. Hence $(x^{(i)})^T\theta$ is closer to zero and hence the probabilites $h_\theta(X)=\text{sigmoid}(X\theta)$ are closer to $\frac{1}{2}$ and further away from $0$ and $1$.

In [11]:
def assess_reg_on_calibration(lambdap = .0000001):
    thetas_b_reg, iters = logistic_regression(Xb, yb0, init_theta=np.zeros(Xb.shape[1]), learning_rate = 1, regularization=lambdap, tol=1.5e-4, prntiters=30*1000)
    print("theta for dataset b={} with regularization".format(thetas_b_reg[-1]))

    hr = sigmoid(Xb.dot(thetas_b_reg[-1]))
    cond_hr01 = (hr<1e-3) | (hr>(1-1e-3))
    print("sum_cond_hr01={}\nhr[cond_hr01]={}\n".format(np.sum(cond_hr01), hr[cond_hr01].round(3)))

    cond_hrf = (hr>1e-3) & (hr<(1-1e-3))
    print("sum_cond_hrf={}\nhr[cond_hrf]={}\n".format(np.sum(cond_hrf), hr[cond_hrf].round(3)))

    predr = (np.sign(hr - 0.5) + 1) / 2
    correct = np.sum(predr == yb0)
    total = len(yb0)
    print("correct={}  total={}  accuracy={}\n".format(correct, total, correct / total))

    a0, b0 = .03, .47
    Iabr0 = [i for i, h0 in enumerate((hr>a0) & (hr<b0)) if h0]
    hfr0 = hr[Iabr0].round(3)
    yb0fr0 = yb0[Iabr0]
    print("Iabr0={}\nhfr0={}\nyb0fr0={}".format(Iabr0, hfr0, yb0fr0))
    print("{}=LHS != RHS={}\n\n".format((np.sum(hfr0)/len(Iabr0)).round(3), np.sum(yb0fr0)/len(Iabr0)))

    a1, b1 = .53, .97
    Iabr1 = [i for i, h1 in enumerate((hr>a1) & (hr<b1)) if h1]
    hfr1 = hr[Iabr1].round(3)
    yb0fr1 = yb0[Iabr1]
    print("Iabr1={}\nhfr1={}\nyb0fr1={}".format(Iabr1, hfr1, yb0fr1))
    print("{}=LHS != RHS={}".format((np.sum(hfr1)/len(Iabr1)).round(3), np.sum(yb0fr1)/len(Iabr1)))

In [12]:
assess_reg_on_calibration(lambdap = .0000001)

iters=30000 theta=11.362478582949619 error=0.0006969327543639614 loss=0.04658003997513213
iters=60000 theta=14.533062492278546 error=0.00044928222830453095 loss=0.03736489950285165
iters=90000 theta=16.80021988192887 error=0.0003468422738616794 loss=0.03271530588836661
iters=120000 theta=18.61912412145116 error=0.00028728366268808046 loss=0.029732614816091425
iters=150000 theta=20.15635221257983 error=0.00024711804705268494 loss=0.02760507236781416
iters=180000 theta=21.49529277172141 error=0.00021768699829983597 loss=0.025992128892944107
iters=210000 theta=22.684942228481418 error=0.00019494783751486858 loss=0.024719376525816742
iters=240000 theta=23.757065532258625 error=0.00017671986810832244 loss=0.023686000791013587
iters=270000 theta=24.733676723197338 error=0.0001617061133696941 loss=0.02282876014992684
theta for dataset b=[-76.5127432   76.67263046  76.52849768] with regularization
sum_cond_hr01=83
hr[cond_hr01]=[ 0.  1.  0.  0.  0.  1.  0.  1.  0.  0.  1.  0.  0.  1.  1.  0.  

In [13]:
assess_reg_on_calibration(lambdap = .00001)

iters=30000 theta=8.822164168210566 error=0.00020707325816751322 loss=0.07880641595115935
theta for dataset b=[-26.77262624  27.12848661  26.6266728 ] with regularization
sum_cond_hr01=56
hr[cond_hr01]=[ 1.     0.     0.     0.     0.     0.     0.     0.     0.     0.     0.
  1.     0.     0.     1.     0.     1.     1.     1.     1.     0.     0.
  1.     0.     1.     1.     0.     0.     0.     1.     0.     1.     1.
  0.     1.     1.     1.     0.999  1.     1.     0.     0.     1.     0.
  1.     1.     0.     0.     0.     1.     1.     1.     0.     1.     0.
  1.   ]

sum_cond_hrf=44
hr[cond_hrf]=[ 0.004  0.399  0.314  0.426  0.998  0.997  0.993  0.002  0.995  0.989
  0.048  0.972  0.52   0.804  0.764  0.998  0.999  0.762  0.992  0.991
  0.002  0.359  0.97   0.976  0.979  0.929  0.094  0.018  0.251  0.104
  0.987  0.978  0.987  0.022  0.236  0.99   0.003  0.588  0.867  0.986
  0.997  0.987  0.661  0.123]

correct=100  total=100  accuracy=1.0

Iabr0=[1, 5, 7, 20, 51, 64, 66,

In [14]:
assess_reg_on_calibration(lambdap = .0001)

theta for dataset b=[-12.16908687  12.88848078  11.99526941] with regularization
sum_cond_hr01=26
hr[cond_hr01]=[ 1.     0.     0.     0.     1.     0.     1.     1.     0.999  0.001  1.
  0.     1.     0.999  1.     1.     0.999  0.999  0.     0.     0.999  0.
  0.999  1.     0.999  0.   ]

sum_cond_hrf=74
hr[cond_hrf]=[ 0.102  0.5    0.007  0.478  0.032  0.448  0.956  0.001  0.948  0.009
  0.01   0.934  0.065  0.921  0.909  0.002  0.214  0.029  0.847  0.001
  0.489  0.008  0.647  0.728  0.952  0.988  0.988  0.965  0.026  0.738
  0.998  0.936  0.92   0.08   0.007  0.002  0.033  0.497  0.002  0.871
  0.893  0.996  0.858  0.031  0.998  0.996  0.83   0.978  0.301  0.139
  0.409  0.291  0.914  0.884  0.002  0.904  0.145  0.996  0.36   0.911
  0.996  0.061  0.028  0.004  0.575  0.747  0.908  0.941  0.001  0.92
  0.999  0.564  0.995  0.392]

correct=99  total=100  accuracy=0.99

Iabr0=[0, 6, 7, 14, 20, 46, 49, 58, 64, 65, 66, 67, 74, 78, 83, 99]
hfr0=[ 0.102  0.032  0.448  0.065  0.214  0.0

In [15]:
assess_reg_on_calibration(lambdap = .001)

theta for dataset b=[-5.08956019  5.81274416  5.02340422] with regularization
sum_cond_hr01=0
hr[cond_hr01]=[]

sum_cond_hrf=100
hr[cond_hrf]=[ 0.341  0.547  0.988  0.14   0.038  0.55   0.243  0.485  0.832  0.058
  0.813  0.132  0.143  0.806  0.292  0.017  0.757  0.769  0.074  0.033
  0.391  0.206  0.704  0.993  0.042  0.057  0.501  0.981  0.145  0.576
  0.981  0.684  0.797  0.885  0.898  0.969  0.834  0.053  0.186  0.695
  0.985  0.032  0.953  0.822  0.977  0.787  0.328  0.126  0.078  0.219
  0.967  0.556  0.074  0.747  0.778  0.986  0.935  0.705  0.217  0.958
  0.932  0.732  0.987  0.877  0.459  0.33   0.499  0.437  0.967  0.791
  0.968  0.754  0.081  0.767  0.33   0.032  0.936  0.029  0.451  0.767
  0.931  0.969  0.024  0.246  0.212  0.102  0.967  0.572  0.973  0.968
  0.663  0.781  0.785  0.062  0.804  0.959  0.014  0.539  0.927  0.542]

correct=96  total=100  accuracy=0.96

Iabr0=[0, 3, 4, 6, 9, 11, 12, 14, 18, 19, 20, 21, 24, 25, 28, 37, 38, 41, 46, 47, 48, 49, 52, 58, 64, 65, 67